In [119]:
# run mean OPA 
import os
# add directory where the utils.py file is located
os.chdir('/esarchive/scratch/kgrayson/git/one_pass/')
import xarray as xr 
import importlib
imported_module = importlib.import_module("meanOPA")
importlib.reload(imported_module)
from meanOPA import *
from meanOPA import meanOPA # from module (file.py) import class name 

#importlib.reload(meanOPA)

#filePath = '/esarchive/scratch/kgrayson/git/one_pass/config.yml'

# then run the meanOPA with the config file 
meanOpa = meanOPA()

####  simulating streaming 
#filePath = "/esarchive/scratch/alacima/projects/destination_earth/icon/tas-uas-vas-ngc2009_atm_2d_1h_inst_20200501T000000Z_5km.nc" 

ds = xr.open_dataset(meanOpa.filePath) # open dataset 

for i in range(24): 
#
    ds1 = ds.isel(time=slice(i,i+1)) # slice(start, stop, step), extract 'moving window' which is hourly data
    dm = meanOpa.mean(ds1)


finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving
finished saving


Testing the output against the np.mean version! 

In [41]:
#filePath = "/esarchive/scratch/alacima/projects/destination_earth/icon/tas-uas-vas-ngc2009_atm_2d_1h_inst_20200501T000000Z_5km.nc" 
#ds = xr.open_dataset(filePath) # open dataset 
 
ds2 = ds.vas
axNum = ds2.get_axis_num('time')
npMean = np.mean(ds.vas, axis = axNum, dtype = np.float64)
npMean = np.squeeze(npMean)

In [102]:
dm.time

<xarray.DataArray 'time' (time: 1)>
array(['2020-05-01T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-05-01

In [93]:
np.squeeze(meanOpa.meanCum) - npMean 


<xarray.DataArray 'vas' (lat: 3600, lon: 7200)>
array([[-2.08166817e-17, -2.08166817e-17, -2.08166817e-17, ...,
        -2.08166817e-17, -2.08166817e-17, -2.08166817e-17],
       [ 0.00000000e+00, -2.22044605e-16, -2.22044605e-16, ...,
         0.00000000e+00,  0.00000000e+00, -2.22044605e-16],
       [ 2.22044605e-16,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  2.22044605e-16, -2.22044605e-16],
       ...,
       [-2.22044605e-16,  2.22044605e-16,  0.00000000e+00, ...,
        -2.22044605e-16,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -2.22044605e-16,  2.22044605e-16],
       [-2.22044605e-16,  2.22044605e-16,  2.22044605e-16, ...,
         2.22044605e-16,  0.00000000e+00, -2.22044605e-16]])
Coordinates:
  * lon       (lon) float64 -179.9 -179.9 -179.8 -179.8 ... 179.9 180.0 180.0
  * lat       (lat) float64 -90.0 -89.95 -89.9 -89.85 ... 89.8 89.85 89.9 89.95
    height_2  float64 10.0

In [6]:
%reset

In [95]:
a = meanOpa.meanCum

AttributeError: 'meanOPA' object has no attribute 'meanCum'

In [84]:
b = meanOpa.tempMean

In [85]:
c = meanOpa.dsNp

In [99]:
timeStampList = sorted(ds.time.data) # this method could be very different for a GRIB file, need to understand time stamps 
timeStampPandas = [pd.to_datetime(x) for x in timeStampList]
timeStamp = timeStampPandas[1]

# this is currently using the numpy version of the list, maybe re-write to work on Pandas date time list
# TIMESTEP SHOUND BE KNOWN APRIORI FROM THE MARS REQUEST
timeStep = np.float64(60) # converting timeStep into minutes, float 64
meanFreq = "daily"
#timeStampList[1] - timeStampList[0] # this is assuming there are a few GRIB mesages per data block 

# need to decide what your default time unit is going to be THIS IS CRITICAL FOR NOW CHOOSING MINUTES
timeStep = (timeStep.astype('timedelta64[m]') / np.timedelta64(1, 'm')) # converting timeStep into minutes, float 64

if(meanFreq == "daily"):
    #
    timeStampMin = timeStamp.minute 
    timeStampHour = timeStamp.hour*60 # converting to minutes - 
    timeStampTot = timeStampMin + timeStampHour 

    # first check if the timeStep is less than an hour, in which case need to count the minutes 
    if(timeStep < 60):
        # this will only work if the one hour is wholly divisable by the timestep 
        if(timeStampTot <= timeStep): # this indicates that it's the first, works when comparing float64 to int
            nData = int(24*60/timeStep) # number of elements of data that need to be added to the cumulative mean 
            count = 0
            meanCum = np.zeros((np.size(ds.lat), np.size(ds.lon))) # only initalise cumulative mean if you know this is the first input

    elif(timeStep < 60*24): # time step is less than a day 
        
        # THIS LINE NEEDS TO CHECK IF IT'S THE FIRST DATA ELEMENT COMING IN
        # NEED TO FIND A MORE ROBUST WAY OF DOING THIS
        if(timeStampHour == 0): # this indicates that it's the first, works when comparing float64 to int,
            nData = int(24*60/timeStep) # number of elements of data that need to be added to the cumulative mean 
            count = 0
            meanCum = np.zeros((np.size(ds.lat), np.size(ds.lon))) # only initalise cumulative mean if you know this is the first input


In [103]:
timeStamp.hour

1

In [111]:
pd.to_datetime[(timeStamp.date())]

TypeError: 'function' object is not subscriptable

In [118]:
str(timeStamp)

'2020-05-01 01:15:00'